# Material Loading

Notebook to load material data into the databse

## Imports

In [ ]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Fabrication data

We have to add the parameters of the material.

The name is a compulsory parameter, it has to be always filled.

In [ ]:
#compulsory parameters

name = 'Juan Ignacio'

main_parameters = {'name':name}

Then metadata parameters are optional and can vary depending on the material so its loading to the database is more dinamic.

The metadata list can be empty.

To load metadata parameters we have to fill a list with dictionaries, the dictionaries may have this parameters:

1. key: The name of the parameter

2. value: The value of the parameter

3. type: The units of the parameter

Here we include an example to save the CPT(layer thickness), which units are millimeters and the value is 0.508:

metadata_parameters.append({'key':'CPT', 'value':'0.508', 'type':'mm'})

In [ ]:
#metadata

metadata_parameters = []

#layer thickness
# metadata_parameters.append({'key':'CPT', 'value':'0.508', 'type':'mm'})

## Load into the table

In [43]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"    -- {key}: {value}")

Parameters to be inserted: 
    -- name: Juan Ignacio


In [33]:
table_name = 'materials'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

INSERT INTO materials (name) VALUES ('Juan Ignacio')


## Retrieve id to load metadata

In [ ]:
data = qrs.get_data(table_name)

data

,id_material,name_material
0,1,Juan Ignacio


Now we store the id of the inserted row, to insert its related metadata

In [37]:
row_id = data['id_material'].values[-1]

print(f"Row ID: {row_id}")

Row ID: 1


## Load into the metadata table

In [ ]:
metadata_table_name =  table_name[:-1] + '_metadata'

for attributes in metadata_parameters:

    attributes[table_name[:-1] + '_id'] = row_id

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(attributes.keys())
    values = ', '.join([f"'{v}'" for v in attributes.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

## Make sure insert was correct

In [ ]:
data = qrs.get_data_metadata(table_name)

data

,id_material,name_material
0,1,Juan Ignacio


In [ ]:
conn.close()